In [ ]:
# Load the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import interpolation as inter
from PIL import Image as im
import pickle
import cv2
from tqdm import tqdm
from scipy.ndimage import rotate
import time
import joblib
from sklearn.metrics import accuracy_score,f1_score,classification_report
import os
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
labels = [ 'Scheherazade New' , 'Marhey' , 'Lemonada' , 'IBM Plex Sans Arabic']
image_size = 600
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        plt.axis('off')
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show() 

In [ ]:
# Load the images from fonts-dataset folder
def load_images():
    # Load the images from the fonts-dataset folder
    images_train = []
    labels_train = []
    # Use tqdm to show a progress bar
    for i in tqdm(labels):
        for filename in os.listdir(f'fonts-dataset/{i}'):
            img = cv2.imread(f'fonts-dataset/{i}/{filename}', cv2.IMREAD_GRAYSCALE)
            # img = cv2.resize(img, (image_size, image_size))
            images_train.append(img)
            labels_train.append(i)
    return images_train, labels_train



In [ ]:
# Load the images
X_train, y_train = load_images()
# Change the y_train to numbers
y_train = [labels.index(i) for i in y_train]

In [ ]:
def find_score(arr, angle):
    """
    Find the score of the skew angle to be used in deskewing the image
    
    Args:
    arr: the image array
    angle: the angle to rotate the image by
    
    Returns:
    hist: the histogram of the image
    score: the score of the skew angle
    """
    
    # mode{‘reflect’, ‘grid-mirror’, ‘constant’, ‘grid-constant’, ‘nearest’, ‘mirror’, ‘grid-wrap’, ‘wrap’}
    data = rotate(arr, angle, reshape=False, order=0, mode='constant', cval=0, prefilter=False)
    hist = np.sum(data, axis=1)
    score = np.sum((hist[1:] - hist[:-1]) ** 2)
    return hist, score

def rotate_image(image, angle):
    """
    Rotates an image by a given angle and fills the remaining pixels with white color.

    Args:
        image: A NumPy array representing the input image.
        angle: The rotation angle in degrees.

    Returns:
        A new NumPy array representing the rotated image.
    """
    # Get image height and width
    height, width = image.shape[:2]

    # Compute the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)

    # Perform the rotation and fill the remaining pixels with white color
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(1, 1, 1))

    return rotated_image

def deskew(binary_img):
    """
    Deskew the image
    
    Args:
    binary_img: the binary image
    
    Returns:
    pix: the deskewed image
    """
    bin_img = (binary_img // 255.0)
    # angles to check for skew angle = 45 degrees and 90 degrees and 180
    angles = np.array ([0 , 45 , 90 , 135 , 180 , 225 , 270 , 315])
    scores = []
    for angle in angles:
        hist, score = find_score(bin_img, angle)
        scores.append(score)

    best_score = max(scores)
    best_angle = angles[scores.index(best_score)]
    print('Best angle: {}'.format(best_angle))

    # correct skew
    # data = rotate(bin_img, best_angle, reshape=False, order=0)
    data = rotate_image(bin_img, best_angle)
    img = im.fromarray((255 * data).astype("uint8"))

    pix = np.array(img)
    return pix

In [ ]:
def preprocess(img):
    """
    Preprocess the image
    
    Args:
    img: the image
    
    Returns:
    img: the preprocessed image
    """
    sharpen_kernel = np.array([[0,-1, 0], [-1,5,-1], [0,-1,0]])
    img = cv2.medianBlur(img, 3) # To remove Salt and Pepper noise
    img = cv2.filter2D(img, -1, sharpen_kernel)  # Sharpen the image
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] # Convert the image to binary
    deskewed_img = deskew(img) # Deskew the image
    final_img = cv2.bitwise_not(deskewed_img) if np.mean(deskewed_img) > 127 else deskewed_img # Invert the image if the mean is less than 127 
    return final_img

In [ ]:
# # Preprocess the images
# X_train_preprocess = [preprocess(i) for i in tqdm(X_train)]
    
# # Dump the preprocessed images to a file
# with open('preprocessed_images.pkl', 'wb') as f:
#     pickle.dump(X_train_preprocess, f)

In [ ]:
# Load the preprocessed images
with open('preprocessed_images.pkl', 'rb') as f:
    X_train_preprocess = pickle.load(f)

In [ ]:
import numpy as np
import cv2 as cv


def save_image(img, folder, title):
    cv.imwrite(f'./{folder}/{title}.png', img)

def projection(gray_img, axis:str='horizontal'):
    """ Compute the horizontal or the vertical projection of a gray image """

    if axis == 'horizontal':
        projection_bins = np.sum(gray_img, 1).astype('int32')
    elif axis == 'vertical':
        projection_bins = np.sum(gray_img, 0).astype('int32')

    return projection_bins

In [ ]:
# def preprocess(image):

#     # Maybe we end up using only gray level image.
#     # gray_img = cv.bitwise_not(image) # Invert the image
#     binary_img = cv.threshold(image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]
#     deskewed_img = deskew(binary_img)
#     return deskewed_img


def projection_segmentation(clean_img, axis, cut=15, min_width=20, min_height=30):
    """Segment the image based on the projection profile

    Args:
        clean_img : Preprocessed image
        axis (str): 'horizontal' or 'vertical'
        cut (int, optional): Gap between the segments. Defaults to 3.
        min_width (int, optional): Width of the segment. Defaults to 5.
        min_height (int, optional): Height of the segment. Defaults to 5.

    Returns:
        _type_: _description_
    """
    segments = []
    start = -1
    cnt = 0

    projection_bins = projection(clean_img, axis)
    for idx, projection_bin in enumerate(projection_bins):

        if projection_bin != 0:
            cnt = 0
        if projection_bin != 0 and start == -1:
            start = idx
        if projection_bin == 0 and start != -1:
            cnt += 1
            if cnt >= cut:
                if axis == 'horizontal':
                    # Line segmentation
                    segment = clean_img[max(start-1, 0):idx, :]
                    # if segment.shape[0] >= min_height:                    
                    segments.append(segment)
                elif axis == 'vertical':
                    # Word segmentation
                    segment = clean_img[:, max(start-1, 0):idx]
                    # if segment.shape[1] >= min_width:
                    segments.append(segment)
                cnt = 0
                start = -1
    
    return segments


# Line Segmentation
#----------------------------------------------------------------------------------------
def line_horizontal_projection(image, cut=3):

    # Segmentation    
    lines = projection_segmentation(image, axis='horizontal', cut=cut)
    return lines


# Word Segmentation
#----------------------------------------------------------------------------------------
def word_vertical_projection(line_image, cut=3):
    
    line_words = projection_segmentation(line_image, axis='vertical', cut=cut)
    line_words.reverse()
    return line_words


def extract_words(img, visual=0):

    lines = line_horizontal_projection(img)
    words = []
    
    for idx, line in enumerate(lines):
        
        if visual:
            # Check for the size of the line to be greater than 30
            # if line.shape[0] > 30:
            save_image(line, 'lines', f'line{idx}')

        line_words = word_vertical_projection(line)
        for w in line_words:
            # if len(words) == 585:
            #     print(idx)
            words.append((w, line))
        # words.extend(line_words)

    # breakpoint()
    if visual:
        for idx, word in enumerate(words):
            # check for the size of the word to be greater than 30
            # print (word[0].shape)
            # if word[0].shape[0] < 100 and word[0].shape[1] > 20 :
            save_image(word[0], 'words', f'word{idx}')
    return words

# # Try to extract the words from the preprocessed images
x = 31
p = preprocess(X_train[x])

# show_images([X_train[999],p])

# Extract the words from the preprocessed images
words = extract_words(p, visual=1)
show_images([X_train[x], p])



In [ ]:
# Resize the images to 600x600
X_train_resized = []
for i in tqdm(X_train_preprocess):
    img = cv2.resize(i, (image_size, image_size))
    X_train_resized.append(img)
X_train_resized = np.array(X_train_resized)
    

In [ ]:
# Apply hog for the images
from skimage.feature import hog

X_train_hog = []
for i in tqdm(X_train_resized):
    X_train_hog.append(hog(i, orientations= 8, pixels_per_cell=(32, 32), cells_per_block=(4, 4), block_norm='L2-Hys'))
    
# Print the shape of the hog features
print(X_train_hog[0].shape)

# # Try hog on one image
# hog_image = hog(X_train[0], orientations= 8, pixels_per_cell=(32, 32), cells_per_block=(4,4), block_norm='L2-Hys')
# print(hog_image.shape)

In [ ]:
# Split the data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_hog, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [ ]:
# Make a pipeline for the model which consist of StandardScaler , PCA and the model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Create a pipeline
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.99)),
])

# Fit the pipeline
pipe.fit(X_train, y_train)

In [ ]:
# Transform the data
X_train_transformed = pipe.transform(X_train)
X_test_transformed = pipe.transform(X_test)

# Print the shape of the transformed data
print(X_train_transformed.shape)

In [ ]:
# lin_model = LogisticRegression()

# # Define the hyperparameters
# param_grid = {
#     'C': np.logspace(-4, 4, 30),
#     'penalty': ['l1', 'l2'],
#     'solver': ['liblinear', 'saga', 'lbfgs'],
#     'warm_start': [True, False]
# }

# # Initialize the RandomizedSearchCV
# random_search = RandomizedSearchCV(lin_model, param_distributions=param_grid, n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1)

# # Fit the model
# random_search.fit(X_train_transformed, y_train)
# # cv_results_df = pd.DataFrame(clf_searched.cv_results_)[relevant_columns].round(decimals=3).sort_values(by='rank_test_score')
# # cv_results_df.head(10)
# relevant_columns = ['param_C', 'param_penalty', 'param_solver', 'param_warm_start', 'mean_test_score', 'std_test_score', 'rank_test_score']
# cv_results_df = pd.DataFrame(random_search.cv_results_)[relevant_columns].round(decimals=3).sort_values(by='rank_test_score')
# cv_results_df.head(10)

# # Print the best parameters
# print(random_search.best_params_)

In [ ]:
# Initialize the model and fit the data

# Initialize the model
# {'warm_start': True, 'solver': 'saga', 'penalty': 'l2', 'C': 0.001}

# {'warm_start': True, 'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.0006723357536499335}
model = LogisticRegression(warm_start=True, solver='saga', penalty='l2', C=0.1, random_state=42)
# model = LogisticRegression()

# Fit the model
model.fit(X_train_transformed, y_train)

# Predict the test data
y_pred = model.predict(X_test_transformed)

# Predict the train data
y_pred_train = model.predict(X_train_transformed)

# Print the training accuracy
print(f"Training Accuracy: {accuracy_score(y_train, y_pred_train)}")

# Print the testing accuracy
print(f"Testing Accuracy: {accuracy_score(y_test, y_pred)}")

# Print the f1 score of the training data
print(f"Training F1 Score: {f1_score(y_train, y_pred_train, average='weighted')}")

# Print the f1 score of the testing data
print (f"Testing F1 Score: {f1_score(y_test, y_pred, average='weighted')}")

# Print the classification report of the training data
print(classification_report(y_train, y_pred_train, target_names=labels))

# Print the classification report of the testing data
print(classification_report(y_test, y_pred, target_names=labels))

# Save the model
joblib.dump(model, 'logistic_model.pkl')

#ACCUARCY: 92.25%

In [ ]:
svm = SVC()

param_dist = {
    'C': np.logspace(-3, 3, 30), 
    'kernel': ['poly', 'rbf'], 
    'degree': [2, 3, 4],
    'gamma': ['scale', 'auto']
}

clf_searched = RandomizedSearchCV(svm, param_dist, n_iter=100, cv=5, random_state=42, n_jobs=-1, verbose=1)

clf_searched.fit(X_train_transformed, y_train)
relevant_columns = ['param_C', 'param_kernel', 'param_gamma', 'mean_test_score', 'std_test_score', 'rank_test_score']
cv_results_df = pd.DataFrame(clf_searched.cv_results_)[relevant_columns].round(decimals=3).sort_values(by='rank_test_score')
cv_results_df.head(10)

# Best parameters of the SVM
# {C : 20.09233 , kernel : 'rbf', gamma : 'scale' , score : 0.908}

In [ ]:
# Initialize the SVM model
svm = SVC(C=20.09233 , kernel='rbf', gamma='scale', probability=True, random_state=42)

# Fit the SVM on the training data
svm.fit(X_train_transformed, y_train)

# Predict the labels of the test set
y_pred = svm.predict(X_test_transformed)
# Predict the labels of the training set
y_pred_train = svm.predict(X_train_transformed)

# Print the accuracy of the SVM model on the training set
print(f"Accuracy of SVM model on the training set: {accuracy_score(y_train, y_pred_train)*100}")

# Print the accuracy of the SVM model on the test set
print(f"Accuracy of SVM model: {accuracy_score(y_test, y_pred)*100}")

# Print the classification report of the SVM model
print(classification_report(y_train, y_pred_train, target_names=labels))

# Print the classification report of the SVM model
print(classification_report(y_test, y_pred, target_names=labels))

# Save the SVM model as h5 file
joblib.dump(svm, 'svm_model.pkl')

#ACCURACY: 92.375%

In [ ]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
random_forest_clf = RandomForestClassifier(random_state=42)

# Define the hyperparameters distribution (use same ranges as before)
param_dist = {
    'n_estimators': np.arange(200, 500), 
    'min_samples_split':  range(2,51),                    # 2 to 50
    'min_samples_leaf': range(1,21),                      # 1 to 20
    'max_depth': range(5, 51, 5),                              # 5 to 50 with step of 5
    'min_impurity_decrease': np.linspace(0.0, 0.2, 20),                   # Decide a reasonable range here (with 20 values)
}

# Initialize RandomizedSearchCV
clf_searched = RandomizedSearchCV(random_forest_clf, param_dist, n_iter=100, cv=5, random_state=42, n_jobs=-1, verbose=1)

clf_searched.fit(X_train_transformed, y_train)
relevant_columns = ['param_n_estimators', 'param_min_samples_split', 'param_min_samples_leaf', 'param_max_depth', 'param_min_impurity_decrease', 
                    'mean_test_score', 'std_test_score', 'rank_test_score']
cv_results_df = pd.DataFrame(clf_searched.cv_results_)[relevant_columns].round(decimals=3).sort_values(by='rank_test_score')
cv_results_df.head(10)

# Best parameters of the Random Forest Classifier
# {'n_estimators': 371, 'min_samples_split': 48, 'min_samples_leaf': 4, 'min_impurity_decrease': 0.0, 'max_depth': 30}
# Best score: 0.776

In [ ]:
random_forest_clf = RandomForestClassifier(n_estimators=371, min_samples_split=48, min_samples_leaf=4, min_impurity_decrease=0.0, max_depth=30, random_state=42)

# Fit the Random Forest Classifier
random_forest_clf.fit(X_train_transformed, y_train)

# Predict the labels of the test set
y_pred = random_forest_clf.predict(X_test_transformed)

# Predict the labels of the training set
y_pred_train = random_forest_clf.predict(X_train_transformed)

# Print the accuracy of the Random Forest Classifier model on the training set
print(f"Accuracy of Random Forest Classifier model on the training set: {accuracy_score(y_train, y_pred_train)*100}")

# Print the accuracy of the Random Forest Classifier model on the test set
print(f"Accuracy of Random Forest Classifier model: {accuracy_score(y_test, y_pred)*100}")

# Print the classification report of the Random Forest Classifier model
print(classification_report(y_train, y_pred_train, target_names=labels))

# Print the classification report of the Random Forest Classifier model
print(classification_report(y_test, y_pred, target_names=labels))

# Save the Random Forest Classifier model as h5 file
joblib.dump(random_forest_clf, 'random_forest_clf.pkl')

#ACCURACY: 80.75%